# <center>detritalPy-mix</center>
### <center>Glenn R. Sharman and Sam A. Johnstone </center>
#### <center>Code for conducting 'top-down', or forward, mixture modeling of detrital zircon U-Pb ages that incorporates a bootstrapping-derived estimate of uncertainty</center>

#### <center>Version 0.1 (Beta)</center>
##### <center>Updated Feb 9, 2024</center>
##### <center><i>Requires detritalPy v1.4.0 or later</i></center>

<center>This is a <b>Beta</b> release detritalPy-mix. Please contact Glenn Sharman (gsharman@uark.edu) if you encounter any errors or problems.</center>

<center>If you find this code helpful in your research, please consider citing Malkowski et al. (2019): American Journal of Science (https://doi.org/10.2475/10.2019.02) and/or Sharman and Johnstone (2017): Earth and Planetary Science Letters (https://doi.org/10.1016/j.epsl.2017.07.044) </center>

### I. Import required modules
This step must by run initially, but then does not need to be run again for the remainder of the analysis session.
<i>Hint: select a cell with code and Shift+Enter to execute it</i>

In [ ]:
import detritalpy
from detritalpy import detritalFuncs as dFunc
from detritalpy import detritalMixer as dMix
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # For improving matplotlib figure resolution
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42
print('detritalPy version: ',detritalpy.__version__)

### II. Import the dataset as an Excel file
This step must by run initially, and should be repeated if any changes are made to the dataset in Excel. The simplest way to load data is to place the Excel file(s) in the same directory or a sub-directory of the Jupyter Notebook file. For more options, please refer to dataLoading.ipynb.

Parent samples are those that will be mixed together. Child samples are the samples that are assumed to be mixtures of the parents. Individual samples or groups of samples can be selected by entering their unique Sample ID's in an array or tuple (see examples below for the correct syntax).

##### Example 1: Sundell and Saylor (2017) dataset 2
The example dataset is described in Saylor and Sundell (2017): Geochemistry, Geophysics, Geosystems (doi.org/10.1002/2016GC006774). A synthetic mixture is derived from mixing 10 sources at known proportions: Source 1 = 30%, Source 2 = 0%, Source 3 = 5%, Source 4 = 5%, Source 5 = 10%, Source 6 = 25%, Source 7 = 5%, Source 8 = 15%, Source 9 = 0%, Source 10 = 5%.

In [ ]:
dataToLoad = ['example-data/Sundell and Saylor (2017) ds02.xlsx']
main_df, main_byid_df, samples_df, analyses_df = dFunc.loadDataExcel(dataToLoad, dataSheet='U_Th_Pb')

parent_list = ['Source_01','Source_02','Source_03','Source_04','Source_05','Source_06','Source_07','Source_08','Source_09','Source_10']
child_list = ['Mixture']

##### Example 2: Sundell and Saylor (2017) dataset 5
The example dataset is described in Saylor and Sundell (2017): Geochemistry, Geophysics, Geosystems (doi.org/10.1002/2016GC006774). Samples from the Loess Plateau (Licht et al., 2016) are modeled as a mixture of four sources: Mu Us, Central Deserts, Qaidam Basin, and Yellow River. The iterative optimization model using Dmax yielded the following best-fit mixture: Mu Us = 1.0%, Central Deserts = 12.0%, Qaidam Basin = 27.0%, and Yellow River = 60.0%.

In [ ]:
# Specify file paths to data input file(s)
dataToLoad = ['example-data/Sundell and Saylor (2017) ds05.xlsx']

main_df, main_byid_df, samples_df, analyses_df = dFunc.loadDataExcel(dataToLoad, dataSheet='U_Th_Pb')

# Sediment sources, or parents, must be formatted as a tuple, as shown below
parent_list = [(['Mu_Us'], 'Mu_Us'),
             (['Central_Deserts'], 'Central Deserts'),
             (['Qaidam_Basin'], 'Qaidam Basin'),
             (['NE_Tibet_Yellow_River'],'Yellow River')]

# Child samples, or samples assumed to be mixtures of the parents
child_list = ['Loess']

##### Example 3: Blum et al. (2018) Scientific Reports dataset
This dataset is from the Bengal Fan and modern Ganges and Brahmaputra rivers. We model the deep-sea fan deposits as mixtures of these two modern rivers as an illustration of how forward sediment mixing may be applied in stratigraphic successions.

In [ ]:
dataToLoad = ['example-data/Blum et al. (2018) Scientific Reports.xlsx']
main_df, main_byid_df, samples_df, analyses_df = dFunc.loadDataExcel(dataToLoad, mainSheet = 'Samples', dataSheet='U_Th_Pb')

# Samples grouped by age
parent_list = [(['M01_GR'],'Modern Ganges River'),
              (['J03'],'Modern Brahmaputra River')]
               
child_list =  [(['U1451A_4H-6H','U1450A_6F-8F','U1452B_8F','U1453A_11F','U1451A_13F_combined','U1453A_26F'],'Mid-Pleistocene'),
              (['U1452B_38F','U1453_32F','U1449A_29,30,31F_combined'],'Early-Pleistocene'),
              (['U1450A_70F','U1450A_78,79,80F','U1450_98F','U1450A_124F'],'Pliocene'),
              (['U1451A_37F','U1451A_41F','U1451A_47,48,49F_pilot','U1451A_60F','U1451A_66F','U1451B_3X','U1451A_80F','U1451A_102F','U1451B_22R'],'Late Miocene'),
              (['U1451B_41R','U1451B_51_54R','U1451B_62R'],'Early- to Mid-Miocene')]

### Calculate best-fit mixture

**Selecting your definition for 'best-fit'**

detritalPy-mix provides a few options for assessing (and minimizing) the difference between observed and modelled distributions. We refer to these functions as 'objective_metric', you can select from the different values printed below. We suggest reviewing Saylor and Sundell (2016): Geosphere (doi:10.1130/GES01237.1) for an overview of these metrics. Note that the 'ss' option is the square root of the sum of squared residuals bewteen the two empirical CDFs (i.e., L2 norm).

In [ ]:
print('Options for objective_metric available in detritalPy-Mix:')
print(dMix.AVAILABLE_METRICS)

The options below specify how the mixture modeling will be conducted. Please choose these parameters carefully, as the result will depend on the objective metric chosen and the choice of KDE parameters (bandwidth), if using one of the KDE-based objective metrics (e.g., r2-kde).

In [ ]:
# What is the comparison metric to use, could be 'dmax','vmax', 'ss', 'r2-pdp','likeness-pdp', 'similarity-pdp', 'r2-kde','likeness-kde', 'similarity-kde', or 'L1norm'
objective_metric = 'Dmax' # This determines if a PDF of a CDF is used to mix and compare data

# Specify the age range (Myr) over which you want to conduct mixture modeling
x1 = 0 # Strongly recommended value of 0 Ma
x2 = 4500 # Strongly recommended value of 4500 Ma

# If using a KDE-based objective metric (e.g., 'r2-kde'), the mixture modeling results will depend on choice of bandwidth
bw = 10 # Bandwidth of gaussian KDE in Myr (only used if a KDE is constructed to apply the chosen comparison metric)
bw_x = None # X-axis location of bandwidth split (only used if multiple KDE values are specified). Set to None if not using a split bandwidth.

# Specify the interval (Myr) over which distributions are calculated
xdif = 1 # Note: an interval of 1 Myr is recommended 


mix_coeffs_bf, obj_func_val, best_mixed_dist = dMix.find_best_fit_mix(parent_list, child_list, main_byid_df, sigma='1sigma', objective_metric=objective_metric,
                      x1=x1, x2=x2, xdif=xdif, bw=bw, bw_x=bw_x, verbose=True)

Optional: Running the code below will list the best-fit mixture of each of the sources for each child sample

In [ ]:
dMix.print_best_fit_mixture(parent_list, child_list, objective_metric, obj_func_val, mix_coeffs_bf)

<b>Recommended exercise: </b> Try a different objective_metric option to see if you get a different result.

### Construct many bootstrapped estimates of mixing coefficients
Rather than rely on a single best-fit mixture, bootstrapping allows for an estimate of uncertainty in the mixing prediction. Bootstrapping is accomplished by calculating the best-fit mixture multiple times, after resampling with replacement both parent and child sample(s).

<b>Note:</b> 100 iterations takes about a minute. Use fewer boostrapps for testing, and then when you are ready, perform a large number (e.g., 5000) and let it run overnight

In [ ]:
# How many bootstrap iterations should we preform. In practice 1000s is a good idea, but use fewer
# during testing to speed things up
nBootstrapIterations = 100

# Do we want to only resample with replacement? Or should original observed ages additionally be redrawn
# based on the observed error
doPerturbResampledAges = False

# Do we want to construct bootstrapped estimates by drawing a specific number of grains from each sample?
# If None, defaults to the observed number of grains
nGrainsToResample = None

# Uncertainly level of input data (options: '1sigma' or '2sigma')
sigma = '1sigma' 

# Perform bootstrapped mixture modelling for each of the samples
mix_coeffs_all, obj_vals_all, child_modelled_distributions = dMix.bootstrap_solve_mixture_models(parent_list,
                                                                           child_list,main_byid_df,
                                                                            sigma = sigma,
                                                                            objective_metric = objective_metric,
                                                                            nBootstrapIterations = nBootstrapIterations, 
                                                                            doPerturbResampledAges = doPerturbResampledAges,
                                                                            nGrainsToResample = nGrainsToResample,
                                                                            x1=x1, x2=x2, xdif=xdif,bw = bw, bw_x = bw_x,
                                                                                                verbose = True, update_freq = 10)

# Create a bootstrapped distribution of the children as well
selfCompMetrics_bs_set, childDists_bs_set = dMix.bootstrapped_self_comparisons_many_samples(main_byid_df,child_list,
                                                                                            doPerturbResampledAges = doPerturbResampledAges,
                                                                                            objective_metric = objective_metric,sigma = sigma,
                                                                                            x1 = x1, x2 = x2,xdif = xdif, bw = bw, bw_x = bw_x,
                                                                                            nBootstrapIterations = nBootstrapIterations)

# Calculate whether the best-fit mixture is a 'good' mixture by comparing the best-fit metric
# with the 5% poorest objective functions from resampling the child
obj_func_crit, worse_than_crit = dMix.calc_model_fit(child_list, obj_func_val, obj_vals_all, selfCompMetrics_bs_set, objective_metric=objective_metric)

### Plot the results of mixture modeling as both cumulative and relative probability distributions
The code below plots the parents and children along with the best-fit or average mixture. Setting plotMixResults = True also shows mixing results as a line or violin plot (not recommended when using a large number of parents)

In [ ]:
plotType = 'KDE' # Specify the type of plot to make: options are 'KDE' or 'PDP'

fillParent = True # Color parents according to their specified color
parent_colors = 'Default' # 'Default' or specify colors as a list (e.g., ['gray','orange','red'])
child_colors = 'Default' # 'Default' or specify colors as a list or as a single color (e.g., 'gray')

color_by_age = False # Will color KDE or PDP according to age categories if set to True
agebins = [0, 200, 1000, 3000] # Specify  age categories for colored KDE or PDP (only used if color_by_age = True)
agebinsc = ['red','orange','green']
agebinsc_alpha = 0.5

xaxis_1 = 0 # Minimum x-axis value of plot
xaxis_2 = 3000 # Maximum x-axis value of plot

# Specify the plot dimensions
w1 = 8 # width of the KDE or PDP plots
w2 = 4 # width of the mixture modeling plots
c = 4 # height of the CDF panel

# What value should we plot for each mixture? Options: 'best-fit', 'average', 'median'
best_plot_type = 'best-fit'

plotPie = True # Will plot a pie diagram of best-fit, median, or average mixture (specified with 'best_plot_type')

# Specify whether to plot the mixture results (not recommended if using a large number (>>10) of parents)
plotMixResults = True 

plotResultType = 'violin' # Options: 'line' or 'violin'
violin_width = 0.2

fig = dMix.plotMix(main_byid_df, parent_list, child_list, plotType=plotType, bw=bw, bw_x=bw_x, x1=x1, x2=x2, xdif=xdif,
                   fillParent = fillParent, parent_colors=parent_colors, child_colors=child_colors,
                   color_by_age=color_by_age, agebins=agebins, agebinsc=agebinsc, agebinsc_alpha=agebinsc_alpha,
                   xaxis_1=xaxis_1, xaxis_2=xaxis_2, w1=w1, w2=w2, c=c, plotPie=plotPie, plotMixResults=plotMixResults,
                   best_plot_type=best_plot_type, plotResultType=plotResultType, violin_width=violin_width, sigma=sigma,
                  mix_coeffs_all=mix_coeffs_all, mix_coeffs_bf=mix_coeffs_bf, obj_func_val=obj_func_val, best_mixed_dist=best_mixed_dist,
                  obj_vals_all=obj_vals_all, objective_metric=objective_metric)

<b>Recommended exercise:</b> Try making best_fit_plot equal to 'average' or 'median'

In [ ]:
# Save figure as PDF
fig.savefig('dPy-mix_results.pdf')

### Visualize the distribution of mixing results

detritalPy-Mix also contains some functions that allow you to visualize the distributions of model results obtained by bootstrapping.

**Assess how modelled objective functions compare to those produced by random sampling**

This plot compares the distribution of objective functions calculated by the bootstrapped mixture models with the distribution of objective functions calculated by comparing a child sample to a resampled version of itself.  If the mixture modelling does a good job reproducing the observed child sample, then these distributions should have a large amount of overlap.

In [ ]:
#Default to None to uses black. Otherwise specify a list of colors for each child, e.g., ['dodgerblue', 'mediumseagreen']
# or a single color for all models, e.g., 'dodgerblue'
modelled_colors = None

#Default to None to use default color pallette from detritalPy, 
# otherwise specify a list of colors for each child, e.g., ['dodgerblue', 'mediumseagreen']
# or a single color for all models, e.g., 'dodgerblue'
self_compared_colors = None

#Adds a title with summary statistics to each plot
doAddSummaryTitle = True

#Set up the axis to plot on
plotWidth = 5.0 #How wide should plots be?
subplotHeight = 3.0 #About how tall should each subplot be?

# Create the figure for plotting
f,axs = plt.subplots(len(selfCompMetrics_bs_set),1,figsize = (plotWidth,subplotHeight*len(child_list)),sharex = True);

#Plot the distributions of objective function values
axs = dMix.plot_many_bootstrapped_metric_comparisons_model_observations(obj_vals_all,
                                                                  selfCompMetrics_bs_set, objective_metric,child_list,
                                                                        main_byid_df,
                                                                        obj_func_crit, worse_than_crit,
                                                                        obj_func_val,
                                                                        modelled_colors = modelled_colors,
                                                                        self_compared_colors = self_compared_colors,
                                                                        axs = axs, plotWidth = plotWidth, subplotHeight = subplotHeight,
                                                                        doAddSummaryTitle=doAddSummaryTitle,
                                                                        alpha = 0.5, bins = 50)

<b>Note:</b> Ideally, the bootstrapped models and bootstrapped self-comparisons will overlap in a 'good' mixture model. The 'crit' objective metric value is equal to the 95th percentile of the bootstrapped self-comparisons. In the synthetic example (Example 1), we expect the model fits to be about as good as the resampled mixture - so the histograms should have quite a bit of overlap.

In [ ]:
#Save figure as PDF
f.savefig('dPy-mix_objective_function_distributions.pdf')

**Assess how modelled distributions compare to the observed distribution**

This plot compares the range of distributions calculated by the bootstrapped mixture models with the observed distributions. If the mixture modelling does a good job reproducing the observed child sample, then these distributions should look similar.

In [ ]:
xaxis_1 = 0 # Minimum x-axis value of plot
xaxis_2 = 3000 # Maximum x-axis value of plot

#Set up the axis to plot on
plotWidth = 6.0 #How wide should plots be?
subplotHeight = 4.0 #About how tall should each subplot be?

#What interval of bootsrapped estimates do you want to summarize?
confidence_interval = 95.0 

#Get some colors for the children
child_colors = 'Default' # Options: 'Default', a list of colors (e.g., ['black','red'], or a single color (e.g., 'black')

#What color do we want to plot model results as?
model_color = 'black'

#How transparent should filled in bounds be?
fill_alpha = 0.25

#Do we want to plot the range of distributions observed in the self-comparison?
plot_self_comparisons = True

#Create the axes to plot on
f,axs = plt.subplots(len(child_list),1,figsize = (plotWidth,subplotHeight*len(child_list)),sharex = True,sharey = 'col')

#Plot
axs = dMix.plot_child_bootstrappedmodel_distribution_comparison(main_byid_df,
                                                                child_modelled_distributions,
                                                                child_list,
                                                                xaxis_1 = xaxis_1, xaxis_2 = xaxis_2,
                                                                x1 = x1, x2 = x2, xdif = xdif, bw = bw, bw_x=bw_x,
                                                                objective_metric = objective_metric,
                                                                confidence_interval = confidence_interval,
                                                                fill_alpha = fill_alpha,
                                                            plot_self_comparisons = plot_self_comparisons,
                                                                childDists_bs_set = childDists_bs_set,
                                                            axs = axs, plotWidth = plotWidth, subplotHeight = subplotHeight,child_colors = child_colors,
                                                                model_color = model_color,sigma = sigma)

<b>Note:</b> In the synthetic example (Example 1), the observed variability in the Mixture should be pretty similar to the variability in the Model results. In the real-world Example 2, we see that the model uncertainty mostly overlaps with the Loess sample. There is some divergence in the youngest part of the U-Pb age distribution (<~250 Ma). In Example 3, te quality of the mixture modeling is not particularly good and gets progressively worse for older units. Perhaps our choice of parents is not capturing the true sediment sources to the Bengal Fan going back in time?

In [ ]:
#Save figure as PDF
f.savefig('dPy-mix_distribution_comparisons.pdf')

**Assess how mixture coefficients vary in a section (or spatially)**

We can plot how mixing coefficients vary with some other dimension by plotting summaries of the bootstrapped mixing coefficients against another variable; for example height in a strat column or distance along a transport system. 

In [ ]:
separate_parents = True # Set to True to plot each parent in a separate subplot. Set to False to plot all results together in the same plot.

# Define the extra dimension (e.g., spatial or temporal value) for plotting each child
# Note: there needs to be a value in this array for each child
yAxisValues = None #np.array([0.5, 100, 300, 600]) #Set to None to just use evenly spaced values

# Should we flip the x and y axis of the plot, such that the extra dimension (called yAxisValues above)
# is plotted on the horizontal axis?
doFlipXY = False

# Should we add 'error bars' to each point?
do_plot_errorbars = True

# What percentage of mixing coefficients should we summarize?
confidence_interval = 95

# What value should we plot for each mixture? Options: 'best-fit', 'average', 'median'
best_plot_type = 'best-fit'

# Speicy the color of parents
parent_colors = 'Default' # Options: 'Default', a list of colors (e.g., ['black','red'], or a single color (e.g., 'black')
plot_alpha = 0.3 # Plotting transparency

plotWidth = 15.0
plotHeight = 4.0

#Make the plot
f, ax = dMix.plot_bootstrapped_mixturecoefficients_stratigraphy(parent_list, child_list, mix_coeffs_bf, mix_coeffs_all,
                                                            confidence_interval = confidence_interval,
                                                            ax = None, plotWidth = plotWidth,
                                                             plotHeight = plotHeight, parent_colors = parent_colors, 
                                                             do_plot_errorbars = do_plot_errorbars,
                                                            yAxisValues = yAxisValues, doFlipXY = doFlipXY, plot_alpha = plot_alpha,
                                                               separate_parents = separate_parents, best_plot_type = best_plot_type);

<b>Note:</b> If you are only plotting one child sample (e.g., Examples 1 and 2), this plot is not very interesting. Example 3 provides a better example of how this plot could be used.

In [ ]:
#Save figure as PDF
f.savefig('dPy-mix_spatial_comparison.pdf')

### Export results as an Excel file

Results can be exported as an Excel file with multiple worksheets

In [ ]:
file_name = 'dPy-mix_results.xlsx'

dMix.export_results(parent_list, child_list, main_byid_df, objective_metric, xdif, bw, mix_coeffs_bf, 
                    obj_func_val, file_name=file_name, verbose=True, version=detritalpy.__version__,
                    bootstrap=True, nBootstrapIterations=nBootstrapIterations,
                    doPerturbResampledAges=doPerturbResampledAges,nGrainsToResample=nGrainsToResample,
                    mix_coeffs_all=mix_coeffs_all,obj_vals_all=obj_vals_all, obj_func_crit=obj_func_crit,
                    worse_than_crit=worse_than_crit, selfCompMetrics_bs_set=selfCompMetrics_bs_set)